<a href="https://colab.research.google.com/github/PochkaChaiki/gasReserves/blob/main/GasReservesCalculations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Imports

In [ ]:
from dataclasses import dataclass

## **Initial data**

In [ ]:
@dataclass
class Property:
    value: float
    min: float
    max: float
    mean: float
    sigma: float

In [ ]:
area                = Property(value=38_556, min=36_628.20, max=40_483.80, mean=38_556, sigma=3650)
effective_thickness = Property(value=11.10, min=10.50, max=12.15, mean=11.10, sigma=0.87)
porosity_coef       = Property(value=0.09, min=0.01, max=0.13, mean=0.09, sigma=0.01)
gas_saturation_coef = Property(value=0.7, min=0.65, max=0.75, mean=0.70, sigma=0.01)
reservoir_pressure  = Property(value=32.30, min=29.07, max=35.53)
relative_density    = Property(value=0.63, min=0.63, max=0.64)
reservoir_temp      = Property(value=320.49, min=288.44, max=352.54)
permeability        = Property(value=0.75, min=0, max=19.14)